## <font color=red>Assignment 2</font>

Use this Jupyter Notebook for the first part of the assignment. It contains the model for the Transportation Problem from the exercises. We have studied the version that assumes that the total demand equals the total supply.<BR>

However, in real life, most transportation problems are not balanced. To solved unbalanced problems, one can formulate slightly different models: one for problems with total supply > total demand and one for problems with total supply < total demand. Alternatively, we can just simply adjust the data in case we encounter an unbalanced problem and thereby transforming it in an equivalent balanced problem. More specific, we add either a dummy supply node or a dummy demand node, and use the balanced model to solve it.<BR>

Before diving into the questions for the assignment, do not forget to study the balanced version of the Transportation Problem (see exercise Computer Class 2). The Gurobi model is included here again. See the cell below. In Part (A) use the balanced model to solve an unbalanced problem with data being read from an Excel file. In Part (B) use it for a coaching problem similar to the Linear Assignment Problem. This assignment problem concerns selecting four swimmers for three 4x100m Medley teams (Womens, Mens and Mixed team). See if you select the gold winning team from the European Championship in 2022.

<B>Do not submit this Jupyter Notebook. Only screenshots of the added code are required for the assignment.</B>

In [1]:
from gurobipy import *

def TransportationModel(supply, demand, cost):

    depots    = range(len(supply))         # Indices for the depots
    customers = range(len(demand))         # Indices for the customers

    if sum(supply) < sum(demand):
        print("Not balanced: supply (", sum(supply), ") < demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    elif sum(supply) > sum(demand):
        print("Not balanced: supply (", sum(supply), ") > demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    
    m = Model("Transportation Problem")
    x = m.addVars(depots, customers, name = "Flow")          # no need to define as INTEGERs (see week 3)
    
    m.setObjective( quicksum(cost[i,j]*x[i,j] for i in depots for j in customers), GRB.MINIMIZE)
    # m.setObjective( x.prod(cost) , GRB.MINIMIZE)               requires dictionaries
    m.addConstrs( (x.sum(i, "*") == supply[i] for i in depots)   , 'Supply')
    m.addConstrs( (x.sum("*", j) == demand[j] for j in customers), 'Demand')
    m.update()
         
    return m

In [2]:
from gurobipy import *

def TransportationModel(supply, demand, cost):

    depots    = range(len(supply))         # Indices for the depots
    customers = range(len(demand))         # Indices for the customers

    if sum(supply) < sum(demand):
        print("Not balanced: supply (", sum(supply), ") < demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    elif sum(supply) > sum(demand):
        print("Not balanced: supply (", sum(supply), ") > demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    
    m = Model("Transportation Problem")
    x = m.addVars(depots, customers, name = "Flow")          # no need to define as INTEGERs (see week 3)
    
    m.setObjective( quicksum(cost[i,j]*x[i,j] for i in depots for j in customers), GRB.MINIMIZE)
    # m.setObjective( x.prod(cost) , GRB.MINIMIZE)               requires dictionaries
    m.addConstrs( (x.sum(i, "*") == supply[i] for i in depots)   , 'Supply')
    m.addConstrs( (x.sum("*", j) == demand[j] for j in customers), 'Demand')
    m.update()
         
    return m

## <font color=red>(Part A) Use the model with all kinds of data</font>

As in real life, most transportation problems are not balanced, can formulate to similar models: one for problems with total supply > total demand and one for problems with total supply < total demand. Alternatively, we can just simply adjust the data in case we encounter an unbalanced problem. That is, add either a dummy supply node or a dummy demand node.
- (A) do not forget to study the balanced version of the Transportation Problem (see exercise Computer Class 2), 
- (B) use it for an unbalanced problem with data being read from an Excel file,
- (C) and use it for a coaching problem similar to the Linear Assignment Problem. More concrete, four swimmers need to be selected for the three 4x100m Medley teams (Womens, Mens and Mixed team). See if you select the gold winning team from the last European Championship.

### Problem W2.1
In the cell below, data is read from an Excel-file "Transportation.xlsx" for an unbalanced Transportation Problem. Running the cell, you will notice that it will not yield a solution. Modify the code below (not the TransportationModel() function) in order to get an equivalent balanced problem instance and present the solution for the original problem instance.

In [7]:
import pandas as pd
import numpy as np

m = 5
n = 5
c = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = 1, nrows = m, usecols = 'B:F').values.tolist()
a = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = 1, nrows = m, usecols = 'G').values.flatten().tolist()
b = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = m+1, nrows = 1, usecols = 'B:F').values.tolist()[0]

### Adjust the parameters in order to have a balanced problem that can be solved with TransportationModel()\
###########################ADDED##################################################################################

c = np.array(c) 

total_supply = sum(a)
total_demand = sum(b)

if total_supply > total_demand: #case 1
    diff = total_supply - total_demand
    b.append(diff)
    c = np.hstack([c, np.zeros((c.shape[0], 1))])

elif total_demand > total_supply: #case 2
    diff = total_demand - total_supply
    a.append(diff)
    c = np.vstack([c, np.zeros((1, c.shape[1]))])

#####################################################################################################################


print("Cost matrix:")
print(c)
print("Supply:", a)
print("Demand:", b)

TP = TransportationModel(a, b, c)
TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

for var in TP.getVars():
   if abs(var.x) > 1e-6: # only printing non-zeros
       print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))

Cost matrix:
[[ 13.  10.  22.  29.  18.   0.]
 [ 14.  13.  16.  21. 999.   0.]
 [  3.   0. 999.  11.   6.   0.]
 [ 18.   9.  19.  23.  11.   0.]
 [ 30.  24.  34.  36.  28.   0.]]
Supply: [5, 6, 7, 4, 3]
Demand: [3, 5, 4, 5, 6, 2]
Set parameter LogToConsole to value 0
Model has 30 variables, 11 constraints and 60 nonzeros

Flow[0,0] =   3.00
Flow[0,1] =   2.00
Flow[1,2] =   4.00
Flow[1,3] =   2.00
Flow[2,1] =   3.00
Flow[2,3] =   3.00
Flow[2,4] =   1.00
Flow[3,4] =   4.00
Flow[4,4] =   1.00
Flow[4,5] =   2.00


## <font color=red>(Part B) Team selection for the 4x100m Medley </font>

### Problem W2.2
In the same Excel file "Transportation.xlsx", the Personal Records of Dutch swimmers who took part in the European Championships of 2022 are provided. Your task is to form teams for the 4x100m relay (Women, Men and Mixed) based on their PR's, i.e. four individuals are selected such that the total PR-time is minimal.

In the cell below, the Women's team is already determined. Adjust the code in order to find the Men's team and the Mixed team. The latter should consist of two male and two female swimmers (check if you have found the winning team).

Note: the problems could be solved by hand rather easily, but that would not be very interesting. Besides that, in reality also reserve-swimmers can be selected for the qualifying heats and the schedule of the championship should be taken in consideration in order to avoid overloading certain swimmers.

In [10]:
import pandas as pd
from datetime import datetime

m = 13                # Number of Women in TeamNL
n = 5                 # The 5-th event is being a supporter
times = pd.read_excel("Transportation.xlsx", sheet_name = 'Medley', header = None, skiprows = 4, nrows = m, usecols = 'B:E').to_numpy()
    
I = range(m)
J = range(4)
    
a = [1 for i in range(m)]
b = [1 for i in range(n)]
b[n-1] = m - n + 1
c = [[ (60*times[i][j].minute + times[i][j].second + times[i][j].microsecond/1e6) for j in range(n-1)] for i in range(m)] 
[c[i].append(0) for i in range(m)]
c = np.array(c)

print("PB's on the four events")
print(c)
print("Supply:", a, "  Demand", b)

TP = TransportationModel(a, b, c)
# TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

for j in J:
    for i in I:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c[i,j]))

# for var in TP.getVars():
#    if abs(var.x) > 1e-6: # only printing non-zeros
#        print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))

PB's on the four events
[[63.91 74.86 54.6  69.96  0.  ]
 [58.65 90.89 54.48 64.26  0.  ]
 [59.62 81.14 55.99 58.1   0.  ]
 [65.56 89.02 54.57 58.71  0.  ]
 [63.25 72.05 53.24 59.3   0.  ]
 [79.77 66.92 68.3  74.43  0.  ]
 [75.37 69.75 54.05 59.12  0.  ]
 [69.76 69.19 58.29 62.64  0.  ]
 [70.44 83.23 55.47 64.38  0.  ]
 [64.7  76.5  55.69 62.21  0.  ]
 [68.24 83.03 55.35 64.75  0.  ]
 [65.94 89.48 55.83 65.19  0.  ]
 [61.35 74.5  58.33 60.44  0.  ]]
Supply: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   Demand [1, 1, 1, 1, 9]
Model has 65 variables, 18 constraints and 130 nonzeros

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) Ultra 5 245K, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 18 rows, 65 columns and 130 nonzeros (Min)
Model fingerprint: 0x6c3ee5a4
Model has 52 linear objective coefficients
Coefficient statistics:
  Matrix range 

Mens team

In [21]:
m = 10
n = 5
times = pd.read_excel("Transportation.xlsx", sheet_name="Medley", header=None, skiprows=20, nrows=m, usecols="B:E").to_numpy()

I = range(m)
J = range(4)

a = [1 for i in range(m)]
b = [1 for i in range(n)]
b[n-1] = m - n + 1

c = [[60*times[i][j].minute + times[i][j].second + times[i][j].microsecond/1e6 for j in range(n-1)] for i in range(m)]
[c[i].append(0) for i in range(m)]
c = np.array(c)

print("PB's on the four events")
print(c)
print("Supply:", a, "  Demand", b)

TP = TransportationModel(a, b, c)
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

for j in J:
    for i in I:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c[i,j]))

PB's on the four events
[[ 70.45  57.8   53.74  54.74   0.  ]
 [ 68.65  83.2   49.42  55.02   0.  ]
 [ 56.95  87.53  48.99  55.71   0.  ]
 [ 58.16  63.4   48.53  60.46   0.  ]
 [ 65.92  66.33  50.48  53.16   0.  ]
 [ 55.48  74.3   48.86  51.41   0.  ]
 [ 56.88 115.    49.67  61.39   0.  ]
 [ 59.44 102.84  49.68  55.25   0.  ]
 [ 64.67  70.7   49.78  53.68   0.  ]
 [ 59.66  66.92  53.93  55.27   0.  ]]
Supply: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   Demand [1, 1, 1, 1, 6]
Model has 50 variables, 15 constraints and 100 nonzeros

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) Ultra 5 245K, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 15 rows, 50 columns and 100 nonzeros (Min)
Model fingerprint: 0x66304b39
Model has 40 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 1e+02]
  Bou

Mixed Team

In [22]:
times_w = pd.read_excel("Transportation.xlsx", sheet_name="Medley", header=None, skiprows=4, nrows=13, usecols="B:E").to_numpy()
times_m = pd.read_excel("Transportation.xlsx", sheet_name="Medley", header=None, skiprows=20, nrows=10, usecols="B:E").to_numpy()
times = np.vstack([times_w, times_m])

m = 23
n = 5

I = range(m)
J = range(4)

a = [1 for i in range(m)]
b = [1 for i in range(n)]
b[n-1] = m - n + 1

c = [[60*times[i][j].minute + times[i][j].second + times[i][j].microsecond/1e6 for j in range(n-1)] for i in range(m)]
[c[i].append(0) for i in range(m)]
c = np.array(c)

print("PB's on the four events")
print(c)
print("Supply:", a, "  Demand", b)

TP = TransportationModel(a, b, c)

TP.addConstr(
    sum(TP.getVarByName(f"Flow[{i},{j}]") for i in range(13) for j in range(4)) == 2
)

print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

for j in J:
    for i in I:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c[i,j]))

PB's on the four events
[[ 63.91  74.86  54.6   69.96   0.  ]
 [ 58.65  90.89  54.48  64.26   0.  ]
 [ 59.62  81.14  55.99  58.1    0.  ]
 [ 65.56  89.02  54.57  58.71   0.  ]
 [ 63.25  72.05  53.24  59.3    0.  ]
 [ 79.77  66.92  68.3   74.43   0.  ]
 [ 75.37  69.75  54.05  59.12   0.  ]
 [ 69.76  69.19  58.29  62.64   0.  ]
 [ 70.44  83.23  55.47  64.38   0.  ]
 [ 64.7   76.5   55.69  62.21   0.  ]
 [ 68.24  83.03  55.35  64.75   0.  ]
 [ 65.94  89.48  55.83  65.19   0.  ]
 [ 61.35  74.5   58.33  60.44   0.  ]
 [ 70.45  57.8   53.74  54.74   0.  ]
 [ 68.65  83.2   49.42  55.02   0.  ]
 [ 56.95  87.53  48.99  55.71   0.  ]
 [ 58.16  63.4   48.53  60.46   0.  ]
 [ 65.92  66.33  50.48  53.16   0.  ]
 [ 55.48  74.3   48.86  51.41   0.  ]
 [ 56.88 115.    49.67  61.39   0.  ]
 [ 59.44 102.84  49.68  55.25   0.  ]
 [ 64.67  70.7   49.78  53.68   0.  ]
 [ 59.66  66.92  53.93  55.27   0.  ]]
Supply: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   Demand [1, 1, 1, 1, 1